In [1]:
!git clone https://github.com/FangShancheng/ABINet.git

Cloning into 'ABINet'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 75 (delta 13), reused 64 (delta 10), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [2]:
%cd ABINet

/content/ABINet


In [3]:
def _get_model(config):
    import importlib
    names = config.model_name.split('.')
    module_name, class_name = '.'.join(names[:-1]), names[-1]
    cls = getattr(importlib.import_module(module_name), class_name)
    model = cls(config)
    return model

In [4]:
names = 'modules.model_vision.BaseVision'.split('.')
names

['modules', 'model_vision', 'BaseVision']

In [5]:
module_name, class_name = '.'.join(names[:-1]), names[-1]
module_name, class_name

('modules.model_vision', 'BaseVision')

In [6]:
import importlib
#from fastai.vision import *
cls = getattr(importlib.import_module(module_name), class_name)
cls

modules.model_vision.BaseVision

In [27]:
configs = "configs/pretrain_vision_model.yaml"
from utils import Config
config = Config(configs)
print(config)

AttributeError: ignored

In [24]:
!pip install -U PyYAML

In [30]:
import logging
import os
import time

import cv2
import numpy as np
import torch
import yaml
from matplotlib import colors
from matplotlib import pyplot as plt
from torch import Tensor, nn

class Config1(object):

    def __init__(self, config_path, host=True):
        def __dict2attr(d, prefix=''):
            for k, v in d.items():
                if isinstance(v, dict):
                    __dict2attr(v, f'{prefix}{k}_')
                else:
                    if k == 'phase':
                        assert v in ['train', 'test']
                    if k == 'stage':
                        assert v in ['pretrain-vision', 'pretrain-language',
                                     'train-semi-super', 'train-super']
                    self.__setattr__(f'{prefix}{k}', v)

        assert os.path.exists(config_path), '%s does not exists!' % config_path
        with open(config_path) as file:
            #config_dict = yaml.load(file, Loader=yaml.FullLoader)
            config_dict = yaml.load(file)
        with open('configs/template.yaml') as file:
            #default_config_dict = yaml.load(file, Loader=yaml.FullLoader)
            default_config_dict = yaml.load(file)
        __dict2attr(default_config_dict)
        __dict2attr(config_dict)
        self.global_workdir = os.path.join(self.global_workdir, self.global_name)

    def __getattr__(self, item):
        attr = self.__dict__.get(item)
        if attr is None:
            attr = dict()
            prefix = f'{item}_'
            for k, v in self.__dict__.items():
                if k.startswith(prefix):
                    n = k.replace(prefix, '')
                    attr[n] = v
            return attr if len(attr) > 0 else None
        else:
            return attr

    def __repr__(self):
        str = 'ModelConfig(\n'
        for i, (k, v) in enumerate(sorted(vars(self).items())):
            str += f'\t({i}): {k} = {v}\n'
        str += ')'
        return str
configs = "configs/pretrain_vision_model.yaml"
config = Config1(configs)
print(config)

ModelConfig(
	(0): dataset_case_sensitive = False
	(1): dataset_charset_path = data/charset_36.txt
	(2): dataset_data_aug = True
	(3): dataset_eval_case_sensitive = False
	(4): dataset_image_height = 32
	(5): dataset_image_width = 128
	(6): dataset_max_length = 25
	(7): dataset_multiscales = False
	(8): dataset_num_workers = 14
	(9): dataset_one_hot_y = True
	(10): dataset_pin_memory = True
	(11): dataset_smooth_factor = 0.1
	(12): dataset_smooth_label = False
	(13): dataset_test_batch_size = 384
	(14): dataset_test_roots = ['data/evaluation/IIIT5k_3000', 'data/evaluation/SVT', 'data/evaluation/SVTP', 'data/evaluation/IC13_857', 'data/evaluation/IC15_1811', 'data/evaluation/CUTE80']
	(15): dataset_train_batch_size = 384
	(16): dataset_train_roots = ['data/training/MJ/MJ_train/', 'data/training/MJ/MJ_test/', 'data/training/MJ/MJ_valid/', 'data/training/ST']
	(17): dataset_use_sm = False
	(18): global_name = pretrain-vision-model
	(19): global_phase = train
	(20): global_seed = None
	(21